In [3]:
import pandas as pd
from glob import  glob
import json
from tqdm.notebook import trange, tqdm
from sklearn.model_selection import StratifiedKFold
import numpy as np

In [6]:
label_path = glob('./train/annotations/*')

In [7]:
label_path

['./train/annotations/2616369a5acb.json',
 './train/annotations/05a4e1dd189f.json',
 './train/annotations/5a6cca560ac5.json',
 './train/annotations/1d6b573f6615.json',
 './train/annotations/8f8be0bb2e11.json',
 './train/annotations/33010b2d7414.json',
 './train/annotations/ddb28682d200.json',
 './train/annotations/45a49bb945f7.json',
 './train/annotations/849af67a94e0.json',
 './train/annotations/9c5b3a247e5c.json',
 './train/annotations/dfc871248d9c.json',
 './train/annotations/41a7017ba0e2.json',
 './train/annotations/9bc8ca31a4fc.json',
 './train/annotations/2c2b772c581e.json',
 './train/annotations/8ea6d7757cee.json',
 './train/annotations/d09b8971f11e.json',
 './train/annotations/8a747781811c.json',
 './train/annotations/e64f06f1bdfd.json',
 './train/annotations/a4364a2a8b64.json',
 './train/annotations/8ef2ee67c78e.json',
 './train/annotations/977d31ef1e5f.json',
 './train/annotations/cbac3df4ec51.json',
 './train/annotations/e6725fe2336a.json',
 './train/annotations/8d5eeef32fc3

In [8]:
len(label_path)

60578

In [9]:
image_path = glob('./train/images/*')

In [10]:
len(image_path)

60578

In [11]:
image_path = []
source = []
gts = []
types = []
for i in  tqdm(label_path):
    with open(i) as f:
        data = json.loads(f.read())
    image_path.append(i.replace('annotations', 'images').replace('json', 'jpg'))
    source.append(data['source'])
    gts.append(data['data-series'])
    types.append(data['chart-type'])

  0%|          | 0/60578 [00:00<?, ?it/s]

In [12]:
df = pd.DataFrame({'image_path':image_path,
                   'source':source,
                   'gts':gts,
                   'types':types
                   })

In [13]:
df

,image_path,source,gts,types
0,./train/images/2616369a5acb.jpg,generated,"[{'x': 'Guyana', 'y': 448116.9295000548}, {'x'...",vertical_bar
1,./train/images/05a4e1dd189f.jpg,generated,"[{'x': 'Kent', 'y': 563411.6581366623}, {'x': ...",vertical_bar
2,./train/images/5a6cca560ac5.jpg,generated,"[{'x': '1961', 'y': 90.56852096783014}, {'x': ...",line
3,./train/images/1d6b573f6615.jpg,generated,"[{'x': 'ISingapore', 'y': 71.10380507910631}, ...",vertical_bar
4,./train/images/8f8be0bb2e11.jpg,generated,"[{'x': '2002', 'y': 101.70427011548507}, {'x':...",line
...,...,...,...,...
60573,./train/images/a98bfe7505fe.jpg,generated,"[{'x': 'Namibia', 'y': 22.65269718440861}, {'x...",vertical_bar
60574,./train/images/0929103983d3.jpg,generated,"[{'x': 'Jan', 'y': 4.10069068255688}, {'x': 'F...",line
60575,./train/images/af0675ba4b7c.jpg,generated,"[{'x': '0', 'y': 29.093819338422392}, {'x': '2...",line
60576,./train/images/91245c2c262b.jpg,generated,"[{'x': 'Taiwan', 'y': 105.05566522207097}, {'x...",vertical_bar


In [14]:
def preprocess_gts(input):
    output = '<x_start>'
    xs = []
    ys = []
    for i in input:
        xs.append(str(i['x']))
        ys.append(str(i['y']))
    output += ';'.join(xs) + '<x_end> <y_start>' + ';'.join(ys) + '<y_end>'
    return output
df['label'] = df['gts'].apply(lambda x:preprocess_gts(x))


In [15]:
df['gts'].values[1]

[{'x': 'Kent', 'y': 563411.6581366623},
 {'x': 'Sweetwater', 'y': 870244.7235720849},
 {'x': 'Lackawanna', 'y': 558832.0601450888},
 {'x': 'Merrimack', 'y': 687060.803909146},
 {'x': 'Kent', 'y': 233680.60274337226},
 {'x': 'Houston', 'y': 407705.32642316417},
 {'x': 'Spokane', 'y': 655003.6179681317},
 {'x': 'Marion', 'y': 737436.3818164542},
 {'x': 'Berkshire', 'y': 279476.58265910693},
 {'x': 'Kane', 'y': 513036.0802293541},
 {'x': 'Atlantic', 'y': 261158.19069281308},
 {'x': 'St. Louis', 'y': 485558.4922799133},
 {'x': 'Box Elder', 'y': 650424.0199765583},
 {'x': 'Ector', 'y': 842767.1356226441},
 {'x': 'Piscataquis', 'y': 558832.0601450888}]

In [16]:
df['label'].values[1]

'<x_start>Kent;Sweetwater;Lackawanna;Merrimack;Kent;Houston;Spokane;Marion;Berkshire;Kane;Atlantic;St. Louis;Box Elder;Ector;Piscataquis<x_end> <y_start>563411.6581366623;870244.7235720849;558832.0601450888;687060.803909146;233680.60274337226;407705.32642316417;655003.6179681317;737436.3818164542;279476.58265910693;513036.0802293541;261158.19069281308;485558.4922799133;650424.0199765583;842767.1356226441;558832.0601450888<y_end>'

In [17]:
df

,image_path,source,gts,types,label
0,./train/images/2616369a5acb.jpg,generated,"[{'x': 'Guyana', 'y': 448116.9295000548}, {'x'...",vertical_bar,<x_start>Guyana;Haiti;High income...;Honduras;...
1,./train/images/05a4e1dd189f.jpg,generated,"[{'x': 'Kent', 'y': 563411.6581366623}, {'x': ...",vertical_bar,<x_start>Kent;Sweetwater;Lackawanna;Merrimack;...
2,./train/images/5a6cca560ac5.jpg,generated,"[{'x': '1961', 'y': 90.56852096783014}, {'x': ...",line,<x_start>1961;1965;1970;1975;1980;1985;1990;19...
3,./train/images/1d6b573f6615.jpg,generated,"[{'x': 'ISingapore', 'y': 71.10380507910631}, ...",vertical_bar,<x_start>ISingapore;Soloman Islands;South Afri...
4,./train/images/8f8be0bb2e11.jpg,generated,"[{'x': '2002', 'y': 101.70427011548507}, {'x':...",line,<x_start>2002;2003;2004;2005;2006;2007;2008;20...
...,...,...,...,...,...
60573,./train/images/a98bfe7505fe.jpg,generated,"[{'x': 'Namibia', 'y': 22.65269718440861}, {'x...",vertical_bar,<x_start>Namibia;Nepal;Netherlands;New Zealand...
60574,./train/images/0929103983d3.jpg,generated,"[{'x': 'Jan', 'y': 4.10069068255688}, {'x': 'F...",line,<x_start>Jan;Feb;Mar;Apr;May;Jun;Jul;Aug;Sep;O...
60575,./train/images/af0675ba4b7c.jpg,generated,"[{'x': '0', 'y': 29.093819338422392}, {'x': '2...",line,<x_start>0;2;4;6;8;10<x_end> <y_start>29.09381...
60576,./train/images/91245c2c262b.jpg,generated,"[{'x': 'Taiwan', 'y': 105.05566522207097}, {'x...",vertical_bar,<x_start>Taiwan;Tajikistan;Tanzania;Thailand;T...


In [18]:
sfk =  StratifiedKFold(5)
df['fold'] = -1
for idx, (_, test) in enumerate(sfk.split(df, df['types'])):
    df.loc[test, 'fold'] = idx

In [19]:
df

,image_path,source,gts,types,label,fold
0,./train/images/2616369a5acb.jpg,generated,"[{'x': 'Guyana', 'y': 448116.9295000548}, {'x'...",vertical_bar,<x_start>Guyana;Haiti;High income...;Honduras;...,0
1,./train/images/05a4e1dd189f.jpg,generated,"[{'x': 'Kent', 'y': 563411.6581366623}, {'x': ...",vertical_bar,<x_start>Kent;Sweetwater;Lackawanna;Merrimack;...,0
2,./train/images/5a6cca560ac5.jpg,generated,"[{'x': '1961', 'y': 90.56852096783014}, {'x': ...",line,<x_start>1961;1965;1970;1975;1980;1985;1990;19...,0
3,./train/images/1d6b573f6615.jpg,generated,"[{'x': 'ISingapore', 'y': 71.10380507910631}, ...",vertical_bar,<x_start>ISingapore;Soloman Islands;South Afri...,0
4,./train/images/8f8be0bb2e11.jpg,generated,"[{'x': '2002', 'y': 101.70427011548507}, {'x':...",line,<x_start>2002;2003;2004;2005;2006;2007;2008;20...,0
...,...,...,...,...,...,...
60573,./train/images/a98bfe7505fe.jpg,generated,"[{'x': 'Namibia', 'y': 22.65269718440861}, {'x...",vertical_bar,<x_start>Namibia;Nepal;Netherlands;New Zealand...,4
60574,./train/images/0929103983d3.jpg,generated,"[{'x': 'Jan', 'y': 4.10069068255688}, {'x': 'F...",line,<x_start>Jan;Feb;Mar;Apr;May;Jun;Jul;Aug;Sep;O...,4
60575,./train/images/af0675ba4b7c.jpg,generated,"[{'x': '0', 'y': 29.093819338422392}, {'x': '2...",line,<x_start>0;2;4;6;8;10<x_end> <y_start>29.09381...,4
60576,./train/images/91245c2c262b.jpg,generated,"[{'x': 'Taiwan', 'y': 105.05566522207097}, {'x...",vertical_bar,<x_start>Taiwan;Tajikistan;Tanzania;Thailand;T...,4


In [20]:
df.groupby(['fold', 'types'])['types'].count()

fold  types         
0     dot               1026
      horizontal_bar      15
      line              4989
      scatter           2248
      vertical_bar      3838
1     dot               1027
      horizontal_bar      15
      line              4988
      scatter           2248
      vertical_bar      3838
2     dot               1026
      horizontal_bar      15
      line              4988
      scatter           2249
      vertical_bar      3838
3     dot               1026
      horizontal_bar      14
      line              4988
      scatter           2249
      vertical_bar      3838
4     dot               1026
      horizontal_bar      14
      line              4989
      scatter           2249
      vertical_bar      3837
Name: types, dtype: int64

In [21]:
df.to_csv('train_with_fold.csv', index=None)